In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
def get_webpage(url, tag, attr_type, val): # 先寫死 attribute: id
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    if attr_type == "id":
        result = soup.find(tag, id=val)
    elif attr_type == "class":
        result = soup.find_all(tag, class_=val) # class 是保留字, bs4 用 class_
    return result

In [4]:
def get_movie_pages(menuUrl):
    movieList = get_webpage(menuUrl, "div", "class", "release_movie_name")
    moviePages = list()
    for movie in movieList:
        # print(movie.select("a"))
        movieChiName = re.sub(r'\s', '', movie.select("a")[0].text)
        movieEngName = re.sub(r'\s', '', movie.select("a")[1].text)
        movieUrl = movie.select("a")[0]['href']
        moviePages.append([movieChiName, movieEngName, movieUrl])
    return moviePages

In [5]:
def get_movie_info(movieInfo):
    # 分類
    movieClassSec = get_webpage(movieInfo[2], "div", "class", "level_name_box")
    movieInfo.append([re.sub(r'\s', '', movieCls.text) for movieCls in movieClassSec[0].select("a")])
    # 上映次數
    moviePage = get_webpage(movieInfo[2], "div", "class", "movie_intro_info_r")
    movieInfo.append(moviePage[0].find("span").text)
    # 劇情介紹
    movieStory = get_webpage(movieInfo[2], "span", "id", "story")
    movieInfo.append(re.sub(r'\s', '', movieStory.text))
    
    return movieInfo[:2] + movieInfo[3:]

In [ ]:
def main():
    file = open('movie_info.txt', 'w')
    sel = get_webpage("https://movies.yahoo.com.tw/", "ul", "id", "mainmenu").select("a")
    mainmenu = dict()
    for s in sel:
        for t in ['本週新片', '上映中', '即將上映']: 
            if t in s.text: # 是以上這些種類的, 連結存放到 main menu
                mainmenu[t] = s['href']
    movieMenu = list()
    for movie, url in mainmenu.items(): # movieMenu: [[[Name, Name, Url], [Name, Name, Url]], [...]]
        movieMenu.append(get_movie_pages(url))
        # pass
    movieInfoList = list()
    for movieList in movieMenu: # movieList: [[Name, Name, Url], [Name, Name, Url]]
        tmp = list()
        for movie in movieList: # movie: [Name, Name, Url]
            movieInfo = get_movie_info(movie)
            tmp.append(movieInfo)
            # write to file
            for i in range(len(movieInfo)):
                if i == 2:
                    file.write(', '.join(movieInfo[i]) + '\n')
                else:
                    file.write(movieInfo[i] + '\n')
        movieInfoList.append(tmp)
    
    # file.writelines(','.join(movieInfoList))
    file.close()
    # print(movieInfoList)
main()